# Домашнее задание: Расчёт метрики PSI
**Дисциплина:** ETL-процессы  
**Тема:** Управление данными и их качество

**Цель:** Реализовать расчёт метрики Population Stability Index (PSI)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1. Синтетическая генерация данных

In [3]:
# Базовые и новые данные
np.random.seed(123)
data_train = np.random.normal(loc=0.0, scale=1.0, size=1000)
data_score = np.random.normal(loc=0.5, scale=1.2, size=1000)

## 2. Альтернативная реализация PSI

In [5]:
def population_stability_index(ref_data, curr_data, bins=10):
    # Создаём разбиение по квантилям на базе референсных данных
    bin_edges = np.percentile(ref_data, np.linspace(0, 100, bins + 1))

    ref_hist, _ = np.histogram(ref_data, bins=bin_edges)
    curr_hist, _ = np.histogram(curr_data, bins=bin_edges)

    ref_percents = ref_hist / len(ref_data)
    curr_percents = curr_hist / len(curr_data)

    # Защита от деления на ноль и логарифма нуля
    ref_percents = np.where(ref_percents == 0, 1e-5, ref_percents)
    curr_percents = np.where(curr_percents == 0, 1e-5, curr_percents)

    psi_values = (ref_percents - curr_percents) * np.log(ref_percents / curr_percents)
    return np.sum(psi_values)

## 3. Расчёт PSI без и с аномалиями

In [6]:
psi_normal = population_stability_index(data_train, data_score)
print(f"PSI (без аномалий): {psi_normal:.4f}")

PSI (без аномалий): 0.2777


In [7]:
# Добавим сильные выбросы в скоред выборку
anomalies = np.random.normal(loc=5.0, scale=0.5, size=100)
data_score_anomalous = np.concatenate([data_score, anomalies])
psi_anomalous = population_stability_index(data_train, data_score_anomalous)
print(f"PSI (с аномалиями): {psi_anomalous:.4f}")

PSI (с аномалиями): 0.2749


## 4. Вывод
- PSI < 0.1 — стабильность сохраняется.
- PSI > 0.2 — существенные изменения, стоит пересмотреть модель или данные.
- Мы показали, как PSI растёт при добавлении аномалий.